In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
import streamlit

In [2]:
data = pd.read_excel(r"C:\datasets\Food_Display_Table.xlsx")
data

,Food_Code,Display_Name,Portion_Default,Portion_Amount,Portion_Display_Name,Factor,Increment,Multiplier,Grains,Whole_Grains,...,Milk,Meats,Soy,Drybeans_Peas,Oils,Solid_Fats,Added_Sugars,Alcohol,Calories,Saturated_Fats
0,12350000,Sour cream dip,1,0.25,cup,0.25,0.25,1.00,0.04799,0.00000,...,0.00000,0.0000,0.0,0.0,0.00000,105.64850,1.57001,0.0,133.65,7.36898
1,13110100,"Ice cream, regular",1,1.00,cup,1.00,0.25,0.25,0.00000,0.00000,...,0.29393,0.0000,0.0,0.0,0.00000,130.99968,95.20488,0.0,267.33,9.03070
2,13110120,"Ice cream, rich",1,1.00,cup,1.00,0.25,0.25,0.00000,0.00000,...,0.25160,0.0962,0.0,0.0,0.00000,213.06672,96.10340,0.0,368.52,15.28840
3,13110200,Soft serve ice cream,1,1.00,cup,1.00,0.25,0.25,0.00000,0.00000,...,0.38233,0.0000,0.0,0.0,0.00000,170.39808,123.83793,0.0,347.73,11.74670
4,13120100,"Ice cream bar, chocolate covered",1,1.00,bar (bar = 3 fl oz),1.00,0.50,0.50,0.00000,0.00000,...,0.09744,0.0000,0.0,0.0,0.00000,112.32144,45.54761,0.0,169.12,9.58720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009,57301500,"Kashi, puffed",1,1.00,cup,1.00,0.25,0.25,0.84672,0.84672,...,0.00000,0.0000,0.0,0.0,0.00005,0.00000,0.00000,0.0,87.60,0.07920
2010,57303100,Kix cereal,1,1.00,cup,1.00,0.25,0.25,0.67804,0.23782,...,0.00000,0.0000,0.0,0.0,0.00000,0.00000,8.93255,0.0,82.94,0.11000
2011,57303100,Kix cereal,2,1.00,single serving box,1.00,0.50,0.50,0.55476,0.19458,...,0.00000,0.0000,0.0,0.0,0.00000,0.00000,7.30845,0.0,67.86,0.09000
2012,57304100,Life cereal (plain & cinnamon),1,1.00,cup,1.00,0.25,0.25,1.24652,0.80124,...,0.00000,0.0000,0.0,0.0,0.00000,0.00000,32.09710,0.0,165.00,0.35640


In [3]:
data.isnull().sum()

Food_Code               0
Display_Name            0
Portion_Default         0
Portion_Amount          0
Portion_Display_Name    0
Factor                  3
Increment               0
Multiplier              0
Grains                  0
Whole_Grains            0
Vegetables              0
Orange_Vegetables       0
Drkgreen_Vegetables     0
Starchy_vegetables      0
Other_Vegetables        0
Fruits                  0
Milk                    0
Meats                   0
Soy                     0
Drybeans_Peas           0
Oils                    0
Solid_Fats              0
Added_Sugars            0
Alcohol                 0
Calories                0
Saturated_Fats          0
dtype: int64

In [4]:
class KnowledgeBase:
    def __init__(self, data_path):
        # Load the dataset
        self.data = pd.read_excel(data_path)
        self.data['Added_Sugars'] = pd.to_numeric(self.data['Added_Sugars'], errors='coerce')
        
    def get_food_info(self, food_id):
        # Retrieve nutritional information for a specific food item by ID
        return self.data.iloc[food_id]

In [5]:
class QLearningAgent:
    def __init__(self, num_actions, learning_rate=0.1, discount_factor=0.9, epsilon=0.1):
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.q_table = np.zeros((3, num_actions))  # 3 rows for breakfast, lunch, and dinner

    def choose_action(self, meal_type):
        if np.random.uniform(0, 1) < self.epsilon:
            # Explore: choose a random action
            return np.random.choice(self.num_actions)
        else:
            # Exploit: choose the action with the highest Q-value for the given meal type
            return np.argmax(self.q_table[meal_type])

    def update_q_table(self, meal_type, action, reward, next_max_q_value):
        # Update Q-value using the Q-learning update rule
        self.q_table[meal_type, action] += self.learning_rate * (reward + self.discount_factor * next_max_q_value - self.q_table[meal_type, action])

In [12]:
def main():
    # Initialize the knowledge base with the dataset
    knowledge_base = KnowledgeBase(r"C:\datasets\Food_Display_Table.xlsx")

    # Define the number of food items (actions) and meal types
    num_food_items = len(knowledge_base.data)
    agent = QLearningAgent(num_food_items)

    # Simulate training over multiple episodes
    num_episodes = 1000
    meal_types = {'breakfast': 0, 'lunch': 1, 'dinner': 2}  # Map meal types to indices

    for episode in range(num_episodes):
        for meal_type, index in meal_types.items():
            # Choose an action (food item) for the current meal type
            action = agent.choose_action(index)
            food_info = knowledge_base.get_food_info(action)

            # Assume some reward function based on the nutritional information
            reward = -1 if food_info['Added_Sugars'] > 50 else 1  # Simplified reward based on carbs

            # Update the Q-table
            next_max_q_value = np.max(agent.q_table[index])  # Get the max Q-value for updating
            agent.update_q_table(index, action, reward, next_max_q_value)

    # After training, display the best food item recommendation for each meal
    best_choices = {meal: knowledge_base.get_food_info(np.argmax(agent.q_table[index]))['Display_Name'] for meal, index in meal_types.items()}
    print("Recommended food items for each meal:")
    for meal, food in best_choices.items():
        print(f"{meal.title()}: {food}")

if __name__ == "__main__":
    main()

Recommended food items for each meal:
Breakfast: French bread
Lunch: Sour cream dip
Dinner: Sour cream dip
